Set the filename

In [1]:
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1805694.chadmin1.ib0.cheyenne.ucar.edu.200417-103849"
filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1851933.chadmin1.ib0.cheyenne.ucar.edu.200420-102252"

"/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1851933.chadmin1.ib0.cheyenne.ucar.edu.200420-102252"

In [2]:
# Open the file
f = open(filename)

# Read through each line and pull out the unique threadnumbers
A = Array{Int64}(undef,0) # Initialize
for line in eachline(f)
    threadnum = parse(Float64,split(line,":")[1]) # Parse the line for the thread number
    if !(in(threadnum,A)) # If not in the array add it
        push!(A,threadnum)
    end
end

# Close the file
close(f)

Run a couple of checks

In [3]:
# Check all entries are unique
println(A == unique(A))

# Check that the thread numbers are sequential
println(sort!(A) == Array(0:maximum(A)))

true
true


Run through the file again and count the number of times that the laimemory is zero

In [58]:
# Set the known strings we are interested in
string_canopy = "Starting canopy trim"
string_cohort = "Current cohort"
string_laimem = "Starting laimemory"
string_season = "EDPftvarcon_inst%season_decid"
string_stress = "EDPftvarcon_inst%stress_decid"
string_evergreen = "EDPftvarcon_inst%evergreen"
string_optimum = "OPTIMUM_LAIMEM"
string_cumulative = "CUMULATIVE LAI"
string_trimopt = "OPTIMUM TRIM"

# Create a dictionary to store the information
check_laimem_zero = Dict("season"=>0,"stress"=>0,"evergreen"=>0)
check_laimem_nonzero = deepcopy(check_laimem_zero)
check_laiopt_zero = deepcopy(check_laimem_zero)
check_laiopt_nonzero = deepcopy(check_laimem_zero)
check_laicum_zero = deepcopy(check_laimem_zero)
check_laicum_nonzero = deepcopy(check_laimem_zero)
check_trimopt_zero = deepcopy(check_laimem_zero)
check_trimopt_positive = deepcopy(check_laimem_zero)
check_trimopt_negative = deepcopy(check_laimem_zero)

based = Dict("season"=>0,"stress"=>0,"evergreen"=>0)
outputd = Dict("LAI memory"=>Dict("nonzero" => based, "zero" => based),
               "LAI optimum" => Dict("nonzero" => based, "zero" => based),
               "LAI cumulative" => Dict("nonzero" => based, "zero" => based),
               "Trim optimum" => Dict("negative" => based, "zero" => based, "positive" => based))

totallaicount = 0;

In [59]:
outputd

Dict{String,Dict{String,Dict{String,Int64}}} with 4 entries:
  "LAI memory"     => Dict("zero"=>Dict("stress"=>0,"season"=>0,"evergreen"=>0)…
  "LAI optimum"    => Dict("zero"=>Dict("stress"=>0,"season"=>0,"evergreen"=>0)…
  "Trim optimum"   => Dict("zero"=>Dict("stress"=>0,"season"=>0,"evergreen"=>0)…
  "LAI cumulative" => Dict("zero"=>Dict("stress"=>0,"season"=>0,"evergreen"=>0)…

In [31]:
for t in A
#     println(t)    
    laiflag = undef
    seasonflag = 0
    stressflag = 0
    evergreenflag = 0
    
    f = open(filename) # Open the file`
    
    # Iterate through file
    for line in eachline(f)
#         totallinecount += 1
        
        linesplit = split(line,":") # Split into array of strings
        threadnum = parse(Float64,linesplit[1]) # Parse the line for the thread number
        
        # Make sure the line format is threadnum:string:val
        if threadnum == t && length(linesplit) == 3 && isempty(findall(isempty,linesplit))
            if strip(linesplit[2]) == string_laimem
                totallaicount += 1  # Count up the total laimemory loops
                seasonflag = 0
                stressflag = 0
                evergreenflag = 0
                if parse(Float64,strip(linesplit[3])) == 0 # Set the laiflag for the next read
                    laiflag = 0
                else
                    laiflag = 1
                end
            elseif strip(linesplit[2]) == string_season && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    check_laimem_zero["season"] += 1
                else 
                    check_laimem_nonzero["season"] += 1                
                end
                laiflag = undef
                seasonflag = 1
            elseif strip(linesplit[2]) == string_stress && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    check_laimem_zero["stress"] += 1
                else 
                    check_laimem_nonzero["stress"] += 1                
                end
                laiflag = undef
                stressflag = 1
            elseif strip(linesplit[2]) == string_evergreen && parse(Float64,strip(linesplit[3])) == 1
                if laiflag == 0
                    check_laimem_zero["evergreen"] += 1
                else 
                    check_laimem_nonzero["evergreen"] += 1                
                end
                laiflag = undef
                evergreenflag = 1
            elseif strip(linesplit[2]) == string_optimum
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        check_laiopt_zero["stress"] += 1
                    elseif seasonflag == 1
                        check_laiopt_zero["season"] += 1
                    elseif evergreenflag == 1
                        check_laiopt_zero["evergreen"] += 1
                    end
                else
                    if stressflag == 1
                        check_laiopt_nonzero["stress"] += 1
                    elseif seasonflag == 1
                        check_laiopt_nonzero["season"] += 1
                    elseif evergreenflag == 1
                        check_laiopt_nonzero["evergreen"] += 1
                    end
                end
            elseif strip(linesplit[2]) == string_cumulative
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        check_laicum_zero["stress"] += 1
                    elseif seasonflag == 1
                        check_laicum_zero["season"] += 1
                    elseif evergreenflag == 1
                        check_laicum_zero["evergreen"] += 1
                    end
                else
                    if stressflag == 1
                        check_laicum_nonzero["stress"] += 1
                    elseif seasonflag == 1
                        check_laicum_nonzero["season"] += 1
                    elseif evergreenflag == 1
                        check_laicum_nonzero["evergreen"] += 1
                    end
                end
            elseif strip(linesplit[2]) == string_trimopt
                if parse(Float64,strip(linesplit[3])) == 0
                    if stressflag == 1
                        check_trimopt_zero["stress"] += 1
                    elseif seasonflag == 1
                        check_trimopt_zero["season"] += 1
                    elseif evergreenflag == 1
                        check_trimopt_zero["evergreen"] += 1
                    end
                elseif parse(Float64,strip(linesplit[3])) > 0
                    if stressflag == 1
                        check_trimopt_positive["stress"] += 1
                    elseif seasonflag == 1
                        check_trimopt_positive["season"] += 1
                    elseif evergreenflag == 1
                        check_trimopt_positive["evergreen"] += 1
                    end
                elseif parse(Float64,strip(linesplit[3])) < 0
                    if stressflag == 1
                        check_trimopt_negative["stress"] += 1
                    elseif seasonflag == 1
                        check_trimopt_negative["season"] += 1
                    elseif evergreenflag == 1
                        check_trimopt_negative["evergreen"] += 1
                    end
                end
            end
        end
    end
close(f) # Close the file
end

In [32]:
check_laimem_nonzero

Dict{String,Int64} with 3 entries:
  "stress"    => 6363
  "season"    => 35933
  "evergreen" => 0

In [33]:
check_laimem_zero

Dict{String,Int64} with 3 entries:
  "stress"    => 58946
  "season"    => 19355
  "evergreen" => 39700

In [34]:
check_laiopt_nonzero

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [35]:
check_laiopt_zero

Dict{String,Int64} with 3 entries:
  "stress"    => 20587
  "season"    => 6619
  "evergreen" => 0

In [36]:
check_laicum_nonzero

Dict{String,Int64} with 3 entries:
  "stress"    => 23878
  "season"    => 8404
  "evergreen" => 1291

In [37]:
check_laicum_zero

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [40]:
check_trimopt_positive

Dict{String,Int64} with 3 entries:
  "stress"    => 21767
  "season"    => 7454
  "evergreen" => 1274

In [41]:
check_trimopt_zero

Dict{String,Int64} with 3 entries:
  "stress"    => 0
  "season"    => 0
  "evergreen" => 0

In [42]:
check_trimopt_negative

Dict{String,Int64} with 3 entries:
  "stress"    => 2111
  "season"    => 950
  "evergreen" => 17

In [12]:
totallaicount

160297

In [48]:
testdict = Dict()

Dict{Any,Any} with 0 entries

In [50]:
testdict["Optimum Trim"] = Dict("negative" => check_trimopt_negative, "zero" => check_trimopt_zero, "positive" => check_trimopt_positive) 

Dict{String,Dict{String,Int64}} with 3 entries:
  "zero"     => Dict("stress"=>0,"season"=>0,"evergreen"=>0)
  "negative" => Dict("stress"=>2111,"season"=>950,"evergreen"=>17)
  "positive" => Dict("stress"=>21767,"season"=>7454,"evergreen"=>1274)

In [52]:
testdict["Optimum Trim"]

Dict{String,Dict{String,Int64}} with 3 entries:
  "zero"     => Dict("stress"=>0,"season"=>0,"evergreen"=>0)
  "negative" => Dict("stress"=>2111,"season"=>950,"evergreen"=>17)
  "positive" => Dict("stress"=>21767,"season"=>7454,"evergreen"=>1274)